In [1]:
import sys
from pathlib import Path

project_root = Path.cwd().parent
sys.path.append(str(project_root))

In [8]:
import yaml
from src.data_preprocessing.tokenizer import NSLTokenizer

with open("../config/config.yaml", 'r', encoding='utf-8') as f:
    config = yaml.safe_load(f)

tokenizer = NSLTokenizer(config=config)

tokens_1 = tokenizer.tokenize("घर")
tokens_2 = tokenizer.tokenize("कख")

print(f"घर tokens: {tokens_1}")
print(f"कख tokens: {tokens_2}")

Vocabulary built! Size: 53 tokens.
घर tokens: [1, 21, 46, 2]
कख tokens: [1, 17, 19, 2]


In [9]:
import pandas as pd
df = pd.read_csv("../training_dataset/master_metadata.csv")

max_f = df['frames'].max()
mean_f = df['frames'].mean()
p95_f = df['frames'].quantile(0.95)

print(f"Longest Video: {max_f} frames")
print(f"Average Video: {mean_f:.2f} frames")
print(f"95% of videos are shorter than: {p95_f} frames")

Longest Video: 187 frames
Average Video: 57.35 frames
95% of videos are shorter than: 121.0 frames


In [ ]:
from src.inference.inference_engine import NSLGenerator
import numpy as np

gen = NSLGenerator("../experiments/best_model.pth", "../vocab.json")
word1 = gen.generate("अ")
word2 = gen.generate("क")

# Check if the arrays are mathematically identical
diff = np.abs(word1 - word2).sum()
print(f"Total numerical difference between 'अ' and 'क': {diff}")

if diff < 0.01:
    print("❌ The model is outputting identical motion. The Encoder is being ignored.")
else:
    print(f"✅ The model is trying to differentiate, but the movement is too small (Diff: {diff:.4f})")

📖 Vocabulary loaded from vocab.json
✅ Successfully loaded seed pose from: KA_31_61.npz
Total numerical difference between 'अ' and 'क': 2810.6962890625
✅ The model is trying to differentiate, but the movement is too small (Diff: 2810.6963)
